In [56]:
import requests
import numpy as np
from math import log, exp
from scipy.optimize import linear_sum_assignment
import time

In [57]:
url = "https://api.twelvedata.com/price"

def get_price(curr1, curr2, key):
    if curr1 == curr2:
        return 1
    else:
        try:
            resp = requests.request("GET", url=f"{url}?symbol={curr1}/{curr2}&amount=100&apikey={key}").json()['price']
            return float(resp)
        except:
            resp = requests.request("GET", url=f"{url}?symbol={curr2}/{curr1}&amount=100&apikey={key}").json()['price']
            return 1/float(resp)

In [58]:
class trader:
    def __init__(self, name, curr, home_curr, start_amount):
        self.name = name
        self.last_roi = 0
        self.curr = np.array(curr)
        self.volume = self.make_volume(home_curr, start_amount)
        self.fees = 0

    def make_volume(self, home_curr, start_amount):
        volume = dict()
        for c in self.curr:
            volume[c] = 0
        volume[home_curr] = start_amount
        return volume
    
    def get_M(self):

        self.P = np.matrix([[0.0 for _ in self.curr]for _ in self.curr], dtype=float)
        self.M = np.matrix([[0.0 for _ in self.curr]for _ in self.curr], dtype=float)
        C = np.matrix([[False for _ in self.curr]for _ in self.curr], dtype=bool)

        for r, curr1 in enumerate(self.curr):
            for c, curr2 in enumerate(self.curr):
                if C[r, c] == False:
                    p = get_price(curr1, curr2, self.key)
                    if p != 0:
                        self.P[r, c] = float(p) 
                        self.P[c, r] = float(1/p)
                        self.M[r, c] = log(float(p))
                        self.M[c, r] = log(float(1/p))
                    else:
                        self.P[r, c] = 1
                        self.P[c, r] = 1
                        self.M[r, c] = 0.0
                        self.M[c, r] = 0.0
                    C[r, c] = C[c, r] = True  

        return self.M

    def set_key(self, key):
        self.key = key

    def trade(self):
        self.get_M()
        row_ind, col_ind = linear_sum_assignment(self.M, maximize=True)
        
        self.last_roi = exp(self.M[row_ind, col_ind].sum())
        
        if self.last_roi > 1 + self.fees:
            child = {k:v for k,v in zip(row_ind, col_ind)}

            visited = set()
            self.orders(child, visited)

    
    def orders(self, child, visited, p = 0):
        if p not in visited:
            visited.add(p)

            print(f"{self.curr[p]}: {self.volume[self.curr[p]]}", end = " -> ")

            self.volume[self.curr[child[p]]] += self.volume[self.curr[p]] * self.P[p, child[p]]
            self.volume[self.curr[p]] = 0

            print(f"{self.curr[child[p]]}: {self.volume[self.curr[child[p]]]}", end=" ")

            self.orders(child, visited, child[p])
        return



In [59]:
personal = "ef3e1be5126a41c0a9d98b02f4a7ce38"
uni = "b86172a213a240fca42701a5e89e0dbe"

hunter = trader("hunter", ["USD", "BTC", "ETH", "INR"], 'USD', 1000)
hunter.set_key(personal)

i = 1
while True:
    hunter.trade()
    print(f'\n{i}th Trade:', hunter.volume['USD'])
    print(hunter.M)
    i += 1
    time.sleep(60)

USD: 1000 -> BTC: 26118300.0 BTC: 26118300.0 -> ETH: 493897053.0 ETH: 493897053.0 -> INR: 53402124958572.0 INR: 53402124958572.0 -> USD: 682926555815.7963 
1th Trade: 682926555815.7963
[[  0.          10.1703915    7.22838845   4.3592185 ]
 [-10.1703915    0.           2.93969088  14.52935457]
 [ -7.22838845  -2.93969088   0.          11.591034  ]
 [ -4.3592185  -14.52935457 -11.591034     0.        ]]


KeyboardInterrupt: 

In [ ]:
# Initial AUD: 1000
# 1th Trade AUD: 1000.5659519774588
# 2th Trade AUD: 1001.0934585700752
# 3th Trade AUD: 1001.6030933464374
# 4th Trade AUD: 1002.1129875667135
# 5th Trade AUD: 1002.623141362981
# 6th Trade AUD: 1003.0787313365812
# 7th Trade AUD: 1003.3897763583743
# 8th Trade AUD: 1003.790647619748
# 9th Trade AUD: 1005.233792049702
# 10th Trade AUD: 1005.7367672640555
# 11th Trade AUD: 1007.1020607377999
# 12th Trade AUD: 1007.5750680037745
# 13th Trade AUD: 1008.0546994439111
# 14th Trade AUD: 1008.6216758181641
# 15th Trade AUD: 1009.377829920072
# 16th Trade AUD: 1009.9993167668223
# 17th Trade AUD: 1010.8830806181251
# 18th Trade AUD: 1011.8841607251356
# 19th Trade AUD: 1012.7258338835934
# 20th Trade AUD: 1013.3931395792628
# 21th Trade AUD: 1014.2218447599173
# 22th Trade AUD: 1014.5917121249926
# 23th Trade AUD: 1014.9324925467234

In [ ]:
print(get_price("USD", "BTC", personal))

None
